In [45]:
FILE_PATH = "C:\Ambarish\AWSBEDROCK-KOLKATA-COMMUNITYDAY-2025\hybrid\CHAP11-BIOLOGY-CLASS11.pdf"

In [46]:
from pdf_helper import get_pdf_data,get_chunks
import uuid
import qdrant_client.http.models as qmodels
from pprint import pprint

In [25]:
full_text = get_pdf_data(FILE_PATH)

In [26]:
len(full_text)

44885

In [27]:
chunks_list = get_chunks(full_text,chunk_length = 1000)

In [28]:
len(chunks_list[2])

936

In [29]:
pprint(chunks_list[2])

('\n'
 'The principles of photosynthesis as established by Calvin\n'
 'are, at present, being used in studies on renewable resource\n'
 'for energy and materials and basic studies in solar energy\n'
 'research.\n'
 ' Melvin Calvin\n'
 '2024-25\n'
 'PHOTOSYNTHESIS  IN HIGHER PLANTS 133\n'
 'All animals including human beings depend on plants for their food. Have\n'
 'you ever wondered from where plants get their food?  Green plants, in fact,\n'
 'have to make or rather synthesise the food they need and all other '
 'organisms\n'
 'depend on them for their needs. The green plants make or rather synthesise\n'
 'the food they need through photosynthesis and are therefore called '
 'autotrophs.\n'
 'You have alr eady lear nt that the autotr ophic nutrition is found only in '
 'plants\n'
 'and all other organisms that depend on the green plants for food are\n'
 'heterotrophs. Green plants carry out ‘photosynthesis’, a physico-chemical\n'
 'process by which they use light energy to drive the 

In [30]:
from fastembed import TextEmbedding

dense_embedding_model = TextEmbedding("sentence-transformers/all-MiniLM-L6-v2")
dense_embeddings = list(dense_embedding_model.passage_embed(chunks_list))
len(dense_embeddings)

49

In [31]:
dense_embeddings[0].shape

(384,)

In [32]:
dense_embeddings[0][:10]

array([-0.00172417,  0.00883244, -0.04961701, -0.00056398, -0.01023328,
        0.04876783, -0.03705058, -0.00908914,  0.04150331,  0.04587442],
      dtype=float32)

In [33]:
from fastembed import LateInteractionTextEmbedding

late_interaction_embedding_model = LateInteractionTextEmbedding("colbert-ir/colbertv2.0")
late_interaction_embeddings = list(late_interaction_embedding_model.passage_embed(chunks_list))
len(late_interaction_embeddings)

49

In [34]:
from fastembed import SparseTextEmbedding

bm25_embedding_model = SparseTextEmbedding("Qdrant/bm25")
bm25_embeddings = list(bm25_embedding_model.passage_embed(chunks_list))
len(bm25_embeddings)

49

In [35]:
(bm25_embeddings[0])

SparseEmbedding(values=array([1.70925645, 1.39751861, 1.70925645, 1.39751861, 1.39751861,
       1.39751861, 1.70925645, 1.39751861, 1.39751861, 1.39751861,
       1.84655738, 1.39751861, 1.39751861, 1.70925645, 1.39751861,
       1.39751861, 1.39751861, 1.92382579, 1.84655738, 1.39751861,
       1.39751861, 1.39751861, 1.39751861, 1.84655738, 1.39751861,
       1.70925645, 1.70925645, 1.70925645, 1.39751861, 1.70925645,
       1.39751861, 1.70925645, 1.39751861, 1.84655738, 1.39751861,
       1.84655738, 1.39751861, 1.70925645, 1.39751861, 1.39751861,
       1.39751861, 1.39751861, 1.39751861, 1.39751861, 1.39751861,
       1.39751861, 1.39751861, 1.39751861, 1.39751861, 1.39751861,
       1.39751861, 1.39751861, 1.39751861, 1.39751861, 1.39751861]), indices=array([1510964117,  516830072, 1223299326, 1305914956,  193870984,
        123462699,  957769532, 2092953102, 2058513491,  939215365,
        358389376, 1923685331, 1547520896, 1015373659,  743323000,
        389043458, 1866681274

In [36]:
from qdrant_client import QdrantClient, models
import os

client = QdrantClient(url = os.getenv("QDRANT_URL"),
                      api_key=os.getenv("QDRANT_KEY"),timeout=600)

In [37]:
os.getenv("QDRANT_URL")

'https://6036eda8-9f83-4b44-9fbd-9cba856823d6.europe-west3-0.gcp.cloud.qdrant.io'

In [38]:
client.create_collection(
    "HYBRID_COLLECTION",
    vectors_config={
        "all-MiniLM-L6-v2": models.VectorParams(
            size=len(dense_embeddings[0]),
            distance=models.Distance.COSINE,
        ),
        "colbertv2.0": models.VectorParams(
            size=len(late_interaction_embeddings[0][0]),
            distance=models.Distance.COSINE,
            multivector_config=models.MultiVectorConfig(
                comparator=models.MultiVectorComparator.MAX_SIM,
            )
        ),
    },
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(
            modifier=models.Modifier.IDF,
        )
    }
)

UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `HYBRID_COLLECTION` already exists!"},"time":0.037744312}'

In [39]:
payload_list = []
id_list =[]

In [40]:
FILE_PATH = "CHAP11-BIOLOGY-CLASS11.pdf"

In [41]:
for i in range(len(chunks_list)):
    payload = {}
    payload["content"] = chunks_list[i]
    payload["filename"] = FILE_PATH
    payload_list.append(payload)
    id_list.append(str(uuid.uuid4()))

In [42]:
len(payload_list)

49

In [43]:
payload_list[0]

{'content': 'UNIT 4\nThe description of structure and variation of living organisms over a\nperiod of time, ended up as two, apparently irreconcilable perspectives\non biology. The two perspectives essentially rested on two levels of\norganisation of life forms and phenomena. One described at organismic\nand above level of organisation while the second described at cellular\nand molecular level of organisation. The first resulted in ecology and\nrelated disciplines. The second resulted in physiology and biochemistry.\nDescription of physiological processes, in flowering plants as an\nexample, is what is given in the chapters in this unit. The processes of\nphotosynthesis, respiration and ultimately plant growth and\ndevelopment are described in molecular terms but in the context of\ncellular activities and even at organism level. Wherever appropriate,\nthe relation of the physiological processes to environment is also\ndiscussed',
 'filename': 'CHAP11-BIOLOGY-CLASS11.pdf'}

In [44]:
batch_size = 30  # Adjust based on your available memory and performance testing
total_items = len(id_list)
from qdrant_client.models import SparseVector

def format_sparse_vector(sparse_embedding):
    return SparseVector(
        indices=sparse_embedding.indices.tolist(),
        values=sparse_embedding.values.tolist()
    )

# Convert embeddings to correct formats:
dense_embeddings = [embedding for embedding in dense_embeddings]
late_interaction_embeddings = [embedding for embedding in late_interaction_embeddings]
bm25_embeddings = [format_sparse_vector(embedding) for embedding in bm25_embeddings]


for i in range(0, total_items, batch_size):
    new_chunk = min(i + batch_size, total_items)

    try:
        client.upsert(
            collection_name="HYBRID_COLLECTION",
            points=qmodels.Batch(
                ids=id_list[i:new_chunk],
                vectors={
                    "all-MiniLM-L6-v2": dense_embeddings[i:new_chunk],
                    "bm25": bm25_embeddings[i:new_chunk],  # Assuming this is correctly formatted for sparse vectors
                    "colbertv2.0": late_interaction_embeddings[i:new_chunk]
                },
                payloads=payload_list[i:new_chunk]
            ),
            wait=True  # Add wait=True to make sure request is processed
        )
        print(f"Upserted items {i} to {new_chunk}")

    except Exception as e:
        print(f"Error upserting items {i} to {new_chunk}: {e}")
        raise e # Raise exception to stop execution.

print("Upsert complete!")

Upserted items 0 to 30
Upserted items 30 to 49
Upsert complete!
